# Współbieżność (i równoległość)

    Concurrency is about dealing with lots of things at once.
    Parallelism is about doing lots of things at once.
    
    Rob Pike

In [1]:
import threading, time

def thread_func(threadName, delay):
   count = 0
   while count < 5:
      time.sleep(delay)
      count += 1
      print(threadName + "  " + time.ctime(time.time()))
    
t1 = threading.Thread(name='1', target=thread_func, args=['1', 1])
t2 = threading.Thread(name='2', target=thread_func, args=['2', 2])
print("S" + "  " + time.ctime(time.time()))
t1.start()
t2.start()
print("E" + "  " + time.ctime(time.time()))


S  Sat Apr  6 11:14:45 2019
E  Sat Apr  6 11:14:45 2019
1  Sat Apr  6 11:14:46 2019
2  Sat Apr  6 11:14:47 2019
1  Sat Apr  6 11:14:47 2019
1  Sat Apr  6 11:14:48 2019
2  Sat Apr  6 11:14:49 2019
1  Sat Apr  6 11:14:49 2019
1  Sat Apr  6 11:14:50 2019
2  Sat Apr  6 11:14:51 2019
2  Sat Apr  6 11:14:53 2019
2  Sat Apr  6 11:14:55 2019


In [2]:
import threading, time

def thread_func(threadName, delay):
   count = 0
   while count < 5:
      time.sleep(delay)
      count += 1
      print(threadName + "  " + time.ctime(time.time()))
    
t1 = threading.Thread(name='1', target=thread_func, args=['1', 1])
t2 = threading.Thread(name='2', target=thread_func, args=['2', 2])
print("S" + "  " + time.ctime(time.time()))
t1.start()
t2.start()
t1.join()
t2.join()
print("E" + "  " + time.ctime(time.time()))


S  Sat Apr  6 11:18:49 2019
1  Sat Apr  6 11:18:50 2019
2  Sat Apr  6 11:18:51 20191  Sat Apr  6 11:18:51 2019

1  Sat Apr  6 11:18:52 2019
2  Sat Apr  6 11:18:53 2019
1  Sat Apr  6 11:18:53 2019
1  Sat Apr  6 11:18:54 2019
2  Sat Apr  6 11:18:55 2019
2  Sat Apr  6 11:18:57 2019
2  Sat Apr  6 11:18:59 2019
E  Sat Apr  6 11:18:59 2019


In [ ]:













praca domowa uwspółbnieżnić zadanie poniżje
uruchomić gunkcje jako wątek i wstawić join-a.
























In [3]:
import random
mylist = [random.randint(0, 100000000) for k in range(10000000)]

def find_min(l, lo, hi):
    if hi - lo < 10000:
        min = l[lo]
        for i in range(lo, hi):
            if l[i] < min:
                min = l[i]
        return min;
    min_a = find_min(l, lo, int(lo + (hi-lo)/2))
    min_b = find_min(l, int(lo + (hi-lo)/2), hi)
    return min_a if min_a < min_b else min_b

print(find_min(mylist, 0, 100000))

823


Dostęp do współdzielonych danych powinien być synchronizowany, choć niektóre z implementacji Pythona będą działać "poprawnie".

In [29]:
import threading, time

a = 0
b = 0

def thread1():
    global a, b
    a += 5
    b += 5

def thread2():
    global a, b
    a += 10
    b += 10

t = threading.Thread(target = thread2)
t.start()
t = threading.Thread(target = thread1)
t.start()
print(a)
print(b)


15
15


Ręczna synchronizacja jest trudna i często prowadzi do problemów ze współdzieleniem danych takich jak np. zakleszczenia.

In [51]:
import threading, time

alock = threading.Lock()
block = threading.Lock()
a = 0
b = 0

def thread1():
    global a, b
    print('Thread1 acquiring lock a')
    alock.acquire()
    print('Thread1 acquiring lock a - got it!')

    print('Thread1 acquiring lock b')
    block.acquire()
    print('Thread1 acquiring lock b - got it!')
    a += 5
    b += 5

    print('Thread1 releasing both locks')
    block.release()
    alock.release()

def thread2():
    global a, b
    print('Thread2 acquiring lock b')
    block.acquire()
    print('Thread2 acquiring lock b - got it!')

    print('Thread2 acquiring lock a')
    alock.acquire()
    print('Thread2 acquiring lock a - got it!')
    a += 10
    b += 10

    print('Thread2 releasing both locks')
    block.release()
    alock.release()


t = threading.Thread(target = thread2)
t.start()
t = threading.Thread(target = thread1)
t.start()


Thread2 acquiring lock b
Thread2 acquiring lock b - got it!
Thread2 acquiring lock a
Thread2 acquiring lock a - got it!
Thread2 releasing both locks
Thread1 acquiring lock a
Thread1 acquiring lock a - got it!
Thread1 acquiring lock b
Thread1 acquiring lock b - got it!
Thread1 releasing both locks


In [59]:
zmienilem alock i block

SyntaxError: invalid syntax (<ipython-input-59-69e8a4417fcc>, line 1)

In [60]:
import threading, time

alock = threading.Lock()
block = threading.Lock()
a = 0
b = 0

def thread1():
    global a, b
    print('Thread1 acquiring lock a')
    alock.acquire()
    print('Thread1 acquiring lock a - got it!')

    print('Thread1 acquiring lock b')
    block.acquire()
    print('Thread1 acquiring lock b - got it!')
    a += 5
    b += 5

    print('Thread1 releasing both locks')
    block.release()
    alock.release()

def thread2():
    global a, b
    print('Thread2 acquiring lock b')
    alock.acquire()
    print('Thread2 acquiring lock b - got it!')

    print('Thread2 acquiring lock a')
    block.acquire()
    print('Thread2 acquiring lock a - got it!')
    a += 10
    b += 10

    print('Thread2 releasing both locks')
    block.release()
    alock.release()


t1 = threading.Thread(target = thread2)
t1.start()
t2 = threading.Thread(target = thread1)
t2.start()
t1.join()
t2.join()

Thread2 acquiring lock b
Thread2 acquiring lock b - got it!
Thread1 acquiring lock aThread2 acquiring lock a

Thread1 acquiring lock a - got it!Thread2 acquiring lock a - got it!

Thread1 acquiring lock bThread2 releasing both locks

Thread1 acquiring lock b - got it!
Thread1 releasing both locks


Z punktu widzenia programisty aplikacja ma do wykonania pewne zadania, nie interesuje go ich zrównoleglanie.

In [64]:
def task(taskName, delay):
    print("S " + taskName + "  " + time.ctime(time.time()))
    time.sleep(delay)
    print("E " + taskName + "  " + time.ctime(time.time()))
    return taskName

print(task('1', 5))
print(task('2', 5))

S 1  Sat Apr  6 11:41:52 2019
E 1  Sat Apr  6 11:41:57 2019
1
S 2  Sat Apr  6 11:41:57 2019
E 2  Sat Apr  6 11:42:02 2019
2


Python pozwala na tworzenie pul wątków. Wtedy automatycznie zarządza wątkami przetwarzając przekazane zadania.

In [69]:
from concurrent.futures import ThreadPoolExecutor, wait

pool = ThreadPoolExecutor(2)

future = pool.submit(task, *('1', 5))
print(future.done())
time.sleep(6)
print(future.done())
print(future.result())

S 1  Sat Apr  6 11:49:22 2019False

E 1  Sat Apr  6 11:49:27 2019
True
1


Metoda `wait(lista zadań)` czeka aż wszystkie zadania zostaną zrealizowane.

In [71]:
pool = ThreadPoolExecutor()

future1 = pool.submit(task, *('1', 5))
future2 = pool.submit(task, *('2', 5))
tasks = [future1, future2]
wait(tasks)
print(future1.result())
print(future2.result())

S 1  Sat Apr  6 11:49:33 2019
S 2  Sat Apr  6 11:49:33 2019
E 1  Sat Apr  6 11:49:38 2019
E 2  Sat Apr  6 11:49:38 2019
1
2


Zysk z współbieżności w Python zależy od implementacji. Niektóre z nich mają zaimplementowany tzw. GIL (Global Intepreter Lock), który blokuje równoległe wykonanie wątków (poza operacjami typu oczekiwanie na dane), ale dzięki temu zabezpiecza przed błędami synchronizacji. Najlepiej korzystać z dedykowanych bibliotek w przypadku obliczeń.

### Zadanie
Utwórz listę wyników w poniższej implementacji. Wyniki można pobrać za pomocą metody `result()` z każdego zadania.

Sprawdź poniższą implementację używając `%%time` dla różnych wielkości `size`. Sprawdź również wykonanie jednowątkowe.

In [ ]:
import random
mylist = [random.randint(0, 100000000) for k in range(10000000)]

def find_min(l, lo, hi):
    min = l[lo]
    for i in range(lo, hi):
        if l[i] < min:
            min = l[i]
    return min;

pool = ThreadPoolExecutor()
size = 100000

lo = 0
hi = size
tasks = []

while lo < len(mylist):
    tasks.append(pool.submit(find_min, *(mylist, lo, hi)))
    lo += size
    hi += size
    if (hi > len(mylist)):
        hi = len(mylist)

wait(tasks)
print(find_min(solutions, 0, len(solutions)))

In [ ]:
TEST

In [87]:
import random
mylist = [random.randint(0, 100000000) for k in range(10000000)]

In [88]:
def find_min(l, lo, hi):
    min = l[lo]
    for i in range(lo, hi):
        if l[i] < min:
            min = l[i]
    return min;

pool = ThreadPoolExecutor()
size = 100000

lo = 0
hi = size
tasks = []

while lo < len(mylist):
    tasks.append(pool.submit(find_min, *(mylist, lo, hi)))
    lo += size
    hi += size
    if (hi > len(mylist)):
        hi = len(mylist)

wait(tasks)
solutions = list(map(lambda t: t.result(),tasks))
print(find_min(solutions, 0, len(solutions)))

4


In [91]:
#wspolbierzność bardzo wolno podnosil sie proces pythonowy
from concurrent.futures import ProcessPoolExecutor

def find_min(l, lo, hi):
    min = l[lo]
    for i in range(lo, hi):
        if l[i] < min:
            min = l[i]
    return min;

pool = ProcessPoolExecutor()
size = 100000

lo = 0
hi = size
tasks = []

while lo < len(mylist):
    tasks.append(pool.submit(find_min, *(mylist, lo, hi)))
    lo += size
    hi += size
    if (hi > len(mylist)):
        hi = len(mylist)

wait(tasks)
solutions = list(map(lambda t: t.result(),tasks))
print(find_min(solutions, 0, len(solutions)))

4
